In [1]:
#!pip install selenium
#!pip install geonamescache
from helper import *
import selenium
import tweepy
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from IPython.display import clear_output #clears output
import datetime
import time

API_Key ='irXxqVrXin1KsDRM9g6HdF8p8'
API_Key_Secret = 'sLQvb8voyrTDz5S93vlt5jE5RlMi8xBuOQtSq2pgERe2QVawIC'

Bearer_Token = 'AAAAAAAAAAAAAAAAAAAAACsVlAEAAAAADib49u9Asa6Hbts8UwBEdtc0dsk%3D1d8lVaySeRg1CpJQ6HDLSwK6Mj9wJiptj49PAdwMuvW95zJoar'

Access_Token = '1122492746210660352-COptyz3RtF1AVokHw3027iOdbsZZl1'
Access_Token_Secret = 'slfBz8pcgCrUmewZgSgVKsmIFhfCcFuinAx16q5eh1Vvr'

In [2]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)

Client = tweepy.Client(bearer_token=Bearer_Token) 


In [3]:
'''
#tweets_list = api.search_30_day(label = '30day',query = '#blender', fromDate = FROM_DATE,toDate = TO_DATE,maxResults = 10,tweet_mode = "extended")
#tweets_list = tweepy.Cursor(api.search_tweets, "biden", count=100).items(100)


start_date = datetime.datetime.now() - datetime.timedelta(days=14)
end_date = start_date + datetime.timedelta(days=7)

start_date = datetime.datetime.now()
end_date = start_date + datetime.timedelta(days=1)
'''

# Search for tweets within the target countries
usa = '96683cc9126741d1'
canada = "3376992a082d67c7"
great_britain = '6416b8512febefc9'
Australia = '3f14ce28dc7c4566'
new_zealand = "0b039ce18ddf1cb9"

tweets_list = []
#ALL_pages = tweepy.Cursor(api.search_tweets, q="since:{} until:{}".format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")), lang="en", tweet_mode="extended", count = 100).pages(50)
max_count_4_each_country = 2000

#add usa
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ usa + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('Us secured')
time.sleep(10)

#add Canada
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ canada + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('canada secured')
time.sleep(10)

#add "Great" Britian
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ great_britain + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('UK secured')
time.sleep(10)

#add Australia
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ Australia + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('kengooro land secured')
time.sleep(10)

#add new zealand
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ new_zealand + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print("kengooroon't land secured")


'''
#return this if you are using pages again!!!!!
for page in ALL_pages:
    tweets_list.extend(page)
#tweets_list = [t for t in tweets_list if t.in_reply_to_status_id == None]    
'''

trends = getAllTrends()#get trends
print('done')

Us secured
canada secured
UK secured
kengooro land secured
kengooroon't land secured
done


In [4]:
tweets_list = [t for t in tweets_list]
len(tweets_list)

10000

In [5]:
#search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)

#user data

DRIVER = webdriver.Chrome()#the chrome

user_name = [];
#location = [];

city = [];
country = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
acc_age = [] 
respectability = []
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
likes = []
retweet_count = []
is_quote_status = []
date = []
tweet_id = []
hashtags = []
hashtags_count = []
trending_for_area = []
is_using_trend = []
users = [];

print('loading users')

arr = []
i = 1

for t in tweets_list:
    tweet.append(t.full_text);
    
    users.append(t.user);
    
    word_count.append(countWordsInString(t.full_text))
    
    likes.append( t.favorite_count)
    
    retweet_count.append(t.retweet_count)
    
    is_quote_status.append(t.is_quote_status)
    
    date.append(t.created_at.isoformat()[:10])
    
    tweet_id.append(t.id)
    
    
        
    city.append(t.place.name)
    country.append( t.place.country)
    
    loc_trends = getTrendsByLoc(t.place.name,t.place.country,trends)
    trending_for_area.append(loc_trends)
    is_using_trend.append(areTrendUsed(t.full_text,loc_trends))
    
    ht = [t['text'] for t in t.entities['hashtags']]
    ht_count = len(ht)
    
    if ht == []:
        ht = np.nan
        ht_count = 0
        
    hashtags.append(ht)
    hashtags_count.append(ht_count)
    
    
    
    
    
        
for x in users:
    if i%26 == 0:
        print("waiting")
        time.sleep(6)
    j = x._json
    
    #opens the page
    screen_name = j['screen_name']    
    DRIVER.get('https://twitter.com/' + screen_name) 
    
    #get the easy stuff
    user_name.append(screen_name)
    following.append(j['friends_count'])
    followers.append(j['followers_count'])
    
    if(j['followers_count'] != 0):
        respectability.append(float(j['friends_count']) / float(j['followers_count']))
    else:
        respectability.append(0)
        
    joined.append(x.created_at.isoformat()[:10])
    acc_age.append(2023 - x.created_at.year + float(x.created_at.day)/12)
    tweet_count_user.append(x.statuses_count)
    verified.append(x.verified)
    
    
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age.append(np.nan)
    else:
        age.append(tmp)
    
    
    gender.append(getGender(j['description'],j['location'],j['name']))
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

10001
done


In [12]:
max = len(gender)

for i in range(0,max):#rounds the account ages
    acc_age[i] = round(acc_age[i],1)

In [7]:
metaData=['--------' for i in range(max)]
dick = {"name":user_name[:max],
        "age":age[:max],
        'city':city[:max],
        'country':country[:max],
        "gender":gender[:max],
        'acount age':acc_age[:max],
        'total tweets': tweet_count_user[:max],
        "followers":followers[:max],
        "following":following[:max],
        'respectability':respectability[:max],
        'verified' : verified[:max],
        'tweet':tweet[:max],
        'likes':likes[:max],
        'retweets':retweet_count[:max],
        'word count': word_count[:max],
        'is quote':is_quote_status[:max],
        'hashtags':hashtags[:max],
        'hashtag count':hashtags_count[:max],
        'trending': trending_for_area[:max],
        'using trends?':is_using_trend[:max],
        'metaData':metaData[:max],
        'join_date':joined[:max],
        'date':date[:max],
        'tweet id':tweet_id[:max]
       }

df = pd.DataFrame(dick)

In [8]:
df

,name,age,city,country,gender,acount age,total tweets,followers,following,respectability,...,word count,is quote,hashtags,hashtag count,trending,using trends?,metaData,join_date,date,tweet id
0,robertreyes,NaN,Long Beach,United States,NaN,15.2,66267,1498,2286,1.526035,...,13,False,NaN,0,"[Rashford, #ManchesterDerby, #MUNMCI, #MUFC, #...",False,--------,2009-01-15,2023-01-14,1614298706161070084
1,RuFlorEsc,NaN,Troy,United States,NaN,13.3,16742,256,845,3.300781,...,9,False,NaN,0,"[Rashford, #ManchesterDerby, #MUNMCI, #Caturda...",False,--------,2010-08-04,2023-01-14,1614298704713768962
2,tmj_USA_gensci,NaN,Tomball,United States,NaN,13.2,453,163,46,0.282209,...,20,False,"[KellyJobs, KellyServices]",2,"[Rashford, #ManchesterDerby, #MUNMCI, #Caturda...",False,--------,2010-04-03,2023-01-14,1614298704567226368
3,BCSportsDad,NaN,Helotes,United States,NaN,6.8,16269,1575,927,0.588571,...,32,False,NaN,0,"[Rashford, #ManchesterDerby, #MUNMCI, #Caturda...",False,--------,2017-03-09,2023-01-14,1614298702503354368
4,JmJk1108_17,NaN,Moberly,United States,NaN,4.2,3294,938,3144,3.351812,...,14,False,"[jin, Dear_Jin_from_ARMY]",2,"[Rashford, #ManchesterDerby, #MUNMCI, #Caturda...",False,--------,2019-04-03,2023-01-14,1614298700318113792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,KauriRescue,NaN,Auckland,New Zealand,NaN,7.7,339,494,137,0.277328,...,45,False,"[kauri, embroidery, crafts, kaurirescue]",4,"[#MUNMCI, Rashford, #MUFC, #BRIvWEL, #SuperSma...",False,--------,2016-12-08,2023-01-12,1613440328228556803
9995,doppelhorn,NaN,Haldane Bay,New Zealand,NaN,16.1,49831,412,672,1.631068,...,14,False,NaN,0,"[#MUNMCI, Rashford, #MUFC, #BRIvWEL, #SuperSma...",False,--------,2009-02-25,2023-01-12,1613440235576360960
9996,Louisepoppy,NaN,Auckland,New Zealand,NaN,14.5,25294,5564,1830,0.328900,...,7,False,"[Thursday, NewZealand]",2,"[#MUNMCI, Rashford, #MUFC, #BRIvWEL, #SuperSma...",False,--------,2009-04-06,2023-01-12,1613439638471053312
9997,cmalietoabrown,NaN,Auckland,New Zealand,NaN,9.1,61475,1670,1055,0.631737,...,24,True,NaN,0,"[#MUNMCI, Rashford, #MUFC, #BRIvWEL, #SuperSma...",False,--------,2015-04-13,2023-01-12,1613439573815857153


In [9]:
DRIVER.quit()

In [10]:
df.to_csv('data4.csv')

In [11]:
df['using trends?'].value_counts()

False    9286
True      713
Name: using trends?, dtype: int64